In [12]:
import os
from os.path import join
import nibabel as nib
from skimage.measure import label, regionprops
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [13]:
def connected_component_analysis(seg, thr=0.5):
    #print(type(seg), seg.shape, seg.min(), seg.max())
    mask = seg.copy()
    mask[mask > thr] = 255
    mask[mask <= thr] = 0

    lbl = label(mask)
    blobs = regionprops(lbl)
    #raw_bbox_list = [b.bbox for b in blobs]
    #axis_major_length_list = [b.axis_major_length for b in blobs]

    return blobs

def cystregionprops(r):
    area = r.area
    major = round(r.axis_major_length, 3)
    
    try:
        minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
    except ValueError:
        minor = np.nan

    return (area, major, minor)


In [14]:
# Folder path
#seg_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\temporary_folder_cyst_segmentations\allegheny\cyst_segmentation'
#mri_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\allegheny\reoriented'

seg_folder_path = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation'

# List to store file paths
seg_file_paths = []
ids = []

# Iterate over files in the folder
for root, dirs, files in os.walk(seg_folder_path):
    for file in files:
        print(file)
        file_path = os.path.join(root, file)
        if file.startswith(".") : continue
        seg_file_paths.append(file_path)




MCF-24.nii.gz
MCF-11.nii.gz
MCF-27.nii.gz
MCF-10.nii.gz
MCF-15.nii.gz
MCF-31.nii.gz
MCF-42.nii.gz
MCF-30.nii.gz
MCF-06.nii.gz
MCF-41.nii.gz
MCF-14.nii.gz
MCF-05.nii.gz
MCF-18.nii.gz
MCF-04.nii.gz
MCF-22.nii.gz
MCF-20.nii.gz
MCF-26.nii.gz
MCF-09.nii.gz
MCF-16.nii.gz
MCF-01.nii.gz


In [15]:
for i in range(len(seg_file_paths)):
    print(f"pair {i+1}")
    print(seg_file_paths[i-1], "\n")

#print(len(seg_file_paths), len(mri_file_paths))

pair 1
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation/MCF-01.nii.gz 

pair 2
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation/MCF-24.nii.gz 

pair 3
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation/MCF-11.nii.gz 

pair 4
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation/MCF-27.nii.gz 

pair 5
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation/MCF-10.nii.gz 

pair 6
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation/MCF-15.nii.gz 

pair 7
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/mayo_florida_mcf/cyst_segmentation/MCF-31.nii.gz 

pair 8
/data/ayc9699/dataset/pancreas_ipmn_ilkin

In [16]:

def create_df():
    column_names=['Center', 'Patient', 'Pancreas_volume_ml', 'Cyst_volume_ml', 'Panc_Cyst_volume_ml', 'Diagonal_mm', 'Panc_vol_to_Diagonal', 'Num_of_Cysts']
    # Add columns using a for loop

    # Define the number of cysts
    num_cysts = 3
    for i in range(num_cysts):
        area_column_name = f'Cyst_{i+1}_vol_ml'
        major_column_name = f'Cyst_{i+1}_major_mm'
        minor_column_name = f'Cyst_{i+1}_minor_mm'
        column_names.append(area_column_name)
        column_names.append(major_column_name)
        column_names.append(minor_column_name)

    # Create an empty DataFrame
    df = pd.DataFrame(columns = column_names)

    df.head()
    return df

In [17]:

def largest3rs(r_list, voxel_sizes):

    voxel_sizes = [voxel_sizes[2], voxel_sizes[1], voxel_sizes[0]]
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
      
        cov_matrix = r.inertia_tensor

        # Perform eigendecomposition to obtain eigenvectors and eigenvalues
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # Determine the dimension along which the major and minor axes lie
        major_axis_dimension = np.argmax(eigenvalues)
        minor_axis_dimension = np.argmin(eigenvalues)

        # Multiply the measurements by the corresponding voxel sizes
        axis_major_length_mm = r.major_axis_length * voxel_sizes[major_axis_dimension]
        try:
            axis_minor_length_mm = r.minor_axis_length * voxel_sizes[minor_axis_dimension]
        except ValueError:
            axis_minor_length_mm = np.nan
        
        # Use the calculated measurements and axis dimensions as needed
        #print(f"Axis Major Length (mm): {axis_major_length_mm}, Dimension: {major_axis_dimension}")
        #print(f"Axis Minor Length (mm): {axis_minor_length_mm}, Dimension: {minor_axis_dimension}")

        vol_ml = r.area * voxel_sizes[0] * voxel_sizes[1] * voxel_sizes[2]/1000

        #major = round(r.axis_major_length, 3)
        
        axis_major_length_mm = round(axis_major_length_mm, 3)
        axis_minor_length_mm = round(axis_minor_length_mm, 3)
        
        '''
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        '''        
        
        top_3.append((vol_ml, axis_major_length_mm, axis_minor_length_mm))
        #top_3.append((vol_ml, major, minor))

    
    return top_3


In [18]:
'''
def largest3rs(r_list):
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
        area = r.area
        major = round(r.axis_major_length, 3)
        
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        
        top_3.append((area, major, minor))
    
    return top_3
    '''


'\ndef largest3rs(r_list):\n    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)\n    top_3 = []\n    \n    for r in sorted_r[:3]:\n        area = r.area\n        major = round(r.axis_major_length, 3)\n        \n        try:\n            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)\n        except ValueError:\n            minor = np.nan\n        \n        top_3.append((area, major, minor))\n    \n    return top_3\n    '

In [19]:
from skimage.measure import label, regionprops
import numpy as np
def compute_diagonal_length_3D(image, voxel_size_x, voxel_size_y, voxel_size_z):
    # Ensure that image is a binary image
    #print(image.shape)
    assert np.array_equal(image, image.astype(bool)), "Input should be a binary image"

    # Label the image
    labeled_image = label(image)
    #print(np.unique(labeled_image))

    # Compute the properties of the labeled regions
    regions = regionprops(labeled_image)

    # Initialize list to store diagonal lengths
    diagonals = []
    #print("voxel",voxel_size_x, voxel_size_y, voxel_size_z)
    for region in regions:
        # Get the bounding box coordinates
        minp, minr, minc, maxp, maxr, maxc = region.bbox
        # Compute the width, height, and depth of the box
        width = (maxp - minp) * voxel_size_x
        height = (maxr - minr) * voxel_size_y
        depth = (maxc - minc) * voxel_size_z
        #print("debug--------", "depth:", (maxc - minc), "y ekseni:", (maxr - minr),  "x ekseni:", (maxp - minp))
        #print(region.bbox, width, height, depth)

        # Use Pythagorean theorem to compute the diagonal length
        diagonal = np.sqrt(width**2 + height**2 + depth**2)
        diagonals.append(diagonal)

    return diagonals

In [20]:
def get_volume_ml(label, voxel_vol_mm3):
    volume_mm3 = np.sum(label != 0) * voxel_vol_mm3
    volume_ml = volume_mm3 / 1000
    return volume_ml

In [25]:
#fig, axs = plt.subplots(1, 3, figsize=(15, 5))

df = create_df()

for eachpatientind in range(len(seg_file_paths)):
    
    pt_name = seg_file_paths[eachpatientind].split("/")[-1].split(".")[0].split("-")[1]
    print(pt_name)
    #if seg_file_paths[eachpatientind].split("\\")[-1].split("_")[-1] == 'pass.nii': continue
    #if pt_name == '0089': continue
    # Read niftii image and segmentation
    #print(pt_name)
    #img_nib = nib.load(mri_file_paths[eachpatientind])
    seg_nib = nib.load(seg_file_paths[eachpatientind])
    
    #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #print(pt_name, img_nib.shape, seg_nib.shape)

    #if pt_name in ['0140', '0159', '0172']:
        #print(seg_file_paths[eachpatientind].split("\\")[-1])
        #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #else:
    #assert img_nib.header.get_zooms() == seg_nib.header.get_zooms()
    #assert img_nib.shape == seg_nib.shape

    voxel_sizes = seg_nib.header.get_zooms()
    print(voxel_sizes)

    #img = img_nib.get_fdata()
    seg = seg_nib.get_fdata()

    print(np.unique(seg))

    #print('img', img.shape)
    #print('seg', seg.shape)

    cysts = (seg == 5)
    if 2 in np.unique(seg):
        pancreas = (seg == 2)
    elif 4 in np.unique(seg):
        pancreas = (seg == 4)
    elif 1 in np.unique(seg):
        pancreas = (seg == 1)
    labels = sorted(np.unique(seg))
    print("ll",labels)
    if len(labels)==2:
        panc_label = labels[1]
        pancreas = (seg == panc_label)
        cysts = (seg == 9999)

    cysts_b = cysts.astype(int)
    pancreas_b = pancreas.astype(int)
    pancreasandcyst = np.concatenate((cysts_b, pancreas_b))

    # Find Bboxes
    blobs = connected_component_analysis(cysts, thr=0.5)
    num_of_cysts = len(blobs)

    # Calculate the voxel volume (assuming isotropic voxel size)
    voxel_vol_mm3 = voxel_sizes[0] *voxel_sizes[1] * voxel_sizes[2]  # Specify the voxel volume in your desired units
    print(voxel_vol_mm3)

    # Calculate the volume of the segmentation mask
    cysts_volume_ml = get_volume_ml(cysts, voxel_vol_mm3)
    pancreas_volume_ml = get_volume_ml(pancreas, voxel_vol_mm3)

    diagonal_lenght = max(compute_diagonal_length_3D(pancreasandcyst, voxel_sizes[0], voxel_sizes[1], voxel_sizes[2]))
    pancreas_volume_to_diagonal = pancreas_volume_ml / diagonal_lenght

    column_data = ['Mayo Florida MCF', pt_name, pancreas_volume_ml, cysts_volume_ml, pancreas_volume_ml + cysts_volume_ml, diagonal_lenght, pancreas_volume_to_diagonal, num_of_cysts]

    result = largest3rs(blobs, voxel_sizes)
    # Print the result
    for i, props in enumerate(result, start=1):
        #print(f"Top {i}: Area={props[0]}, Major={props[1]}, Minor={props[2]}")
        area, major, minor = props[0], props[1], props[2]
        column_data.append(area)
        column_data.append(major)
        column_data.append(minor)

    column_data.extend([math.nan] * (17 - len(column_data)))
    #print(column_data)

    df.loc[len(df)] = column_data

df

#plt.show()

24
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
11
(1.1875, 1.1875, 6.0)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
8.4609375
27
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
10
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
15
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
31
(0.8008, 0.8008, 6.000155)
[0. 1. 5.]
ll [0.0, 1.0, 5.0]
3.8477833
42
(0.9375, 0.9375, 4.4)
[0. 1. 5.]
ll [0.0, 1.0, 5.0]
3.8671875
30
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
06
(0.90625, 0.90625, 4.4)
[0. 1. 5.]
ll [0.0, 1.0, 5.0]
3.613672
41
(0.90625, 0.90625, 4.4)
[0. 2. 5.]
ll [0.0, 2.0, 5.0]
3.613672
14
(1.5625, 1.5625, 7.8)
[0. 4.]
ll [0.0, 4.0]
19.042969
05
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
18
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
04
(0.90625, 0.90625, 4.4)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
3.613672
22
(1.1875, 1.1875, 6.0)
[0. 4. 5.]
ll [0.0, 4.0, 5.0]
8.4609375
20
(0.90625, 0.906

,Center,Patient,Pancreas_volume_ml,Cyst_volume_ml,Panc_Cyst_volume_ml,Diagonal_mm,Panc_vol_to_Diagonal,Num_of_Cysts,Cyst_1_vol_ml,Cyst_1_major_mm,Cyst_1_minor_mm,Cyst_2_vol_ml,Cyst_2_major_mm,Cyst_2_minor_mm,Cyst_3_vol_ml,Cyst_3_major_mm,Cyst_3_minor_mm
0,Mayo Florida MCF,24,66.744522,7.657371,74.401893,202.950544,0.328871,26,1.420173,25.654,19.354,2.623526,22.751,29.855,0.455323,19.348,8.644
1,Mayo Florida MCF,11,54.666117,10.043133,64.709250,176.682459,0.309403,11,9.400102,112.081,21.944,0.219984,10.327,10.353,0.118453,7.235,0.000
2,Mayo Florida MCF,27,147.770276,101.930847,249.701123,260.751194,0.566710,100,78.994869,69.126,63.797,13.717499,36.211,35.117,1.463537,26.754,15.854
3,Mayo Florida MCF,10,57.916321,15.394243,73.310564,171.685337,0.337340,14,10.067690,53.975,33.517,4.376157,32.892,37.879,0.404731,16.147,6.504
4,Mayo Florida MCF,15,25.595639,1.900791,27.496430,150.677131,0.169871,17,0.430027,21.182,6.434,0.654075,12.732,9.025,0.144547,12.048,7.254
5,Mayo Florida MCF,31,91.177074,4.070955,95.248028,205.026113,0.444710,1,4.070955,18.775,37.669,NaN,NaN,NaN,NaN,NaN,NaN
6,Mayo Florida MCF,42,73.321875,0.189492,73.511367,176.896963,0.414489,10,0.054141,6.900,NaN,0.023203,3.423,0.000,0.015469,2.096,0.000
7,Mayo Florida MCF,30,39.529958,29.520087,69.050045,171.613247,0.230343,7,29.212924,145.346,52.556,0.187911,15.299,NaN,0.072273,8.699,0.000
8,Mayo Florida MCF,06,55.983007,24.038146,80.021153,184.643283,0.303195,6,23.575596,156.321,28.372,0.195138,13.010,NaN,0.101183,10.515,NaN
9,Mayo Florida MCF,41,46.648892,4.462885,51.111777,180.275332,0.258765,15,2.374182,35.298,12.629,0.578188,17.758,10.808,0.487846,14.319,7.678


In [26]:
# Save the DataFrame to a CSV file
df.to_csv('server_cyst_segmentation_mayo_florida_mcf.csv', index=False)